# Training a  Mask R-CNN

Jupyter notebook used to apply a mask R-cnn model in inference mode to classify live maerl in imagery of the sea-bed obtained from drop-down cameras. 
Originally based on https://github.com/matterport/Mask_RCNN Also see this repo for more examples (in samples folder) and explanation of config parameters.
Platform used in this instance : JupyterLab on MAGEOHub (massive online GPU for Earth Observation at Plymouth Marine Lab).
This notebook loads necessary modules , loads images to be classified (imagery in tiff or jpeg )  loads pre-trained weights and outputs classified rasters. This is a binary classification of live maerl/ not live maerl. 
Many thanks to Dan Simms (University of Cranfield) for code examples and Stephen Goult & Dan Clewley  (Plymouth Marine Laboratory/ NEODAAS) for troubleshooting suggestions and use of MAGEO.
Keen to hear about other applications - karen.frake@nature.scot

In [1]:
import sys
sys.version


'3.8.8 | packaged by conda-forge | (default, Feb 20 2021, 16:22:27) \n[GCC 9.3.0]'

!ln -s /lustre_scratch/karennaturescot ~/lustre_scratch

##Data transfer



In [ ]:
!nvidia-smi 


In [2]:
import tensorflow
tensorflow.config.list_physical_devices(
    device_type=None
)


[PhysicalDevice(name='/physical_device:CPU:0', device_type='CPU'),
 PhysicalDevice(name='/physical_device:GPU:0', device_type='GPU')]

In [2]:
import os
#%%bash
os.chdir('/lustre_scratch/karennaturescot')
#os.chdir('/lustre_scratch/karennaturescot/Deep_learning_maerl')
#cd mask_rcnn

In [3]:
pwd

'/lustre_scratch/karennaturescot'

In [4]:
%%bash
#os.chdir('/lustre_scratch/karennaturescot')
#cd mask_rcnn
ls 

Deep_learning_maerl
delete
delete1
karennaturescot
mask_rcnn


In [5]:
import sys
sys.path.append('/lustre_scratch/karennaturescot/mask_rcnn')



In [6]:
pwd

'/lustre_scratch/karennaturescot'

In [7]:
!pip install pip==21.0.1
!pip install keras==2.4.3 # tried 2.1.5 #07 06 20212.4.3
!pip install cython
!pip install opencv-python

distutils: /opt/conda/include/python3.8/UNKNOWN
sysconfig: /opt/conda/include/python3.8
user = False
home = None
root = None
prefix = None
  Using cached pip-21.0.1-py3-none-any.whl (1.5 MB)
  Attempting uninstall: pip
    Found existing installation: pip 21.1
    Uninstalling pip-21.1:
      Successfully uninstalled pip-21.1
distutils: /opt/conda/include/python3.8/UNKNOWN
sysconfig: /opt/conda/include/python3.8
user = False
home = None
root = None
prefix = None
  Using cached Keras-2.4.3-py2.py3-none-any.whl (36 kB)
You should consider upgrading via the '/opt/conda/bin/python3.8 -m pip install --upgrade pip' command.
  Using cached Cython-0.29.24-cp38-cp38-manylinux1_x86_64.whl (1.9 MB)
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
cudf 0.19.1 requires cupy-cuda112, which is not installed.
You should consider upgrading via the '/opt/conda/bin/python3.8 -m pi

In [8]:
from osgeo import gdal

In [9]:
!pip install scikit-image==0.16.2

  Using cached scikit_image-0.16.2-cp38-cp38-manylinux1_x86_64.whl (26.5 MB)
  Attempting uninstall: scikit-image
    Found existing installation: scikit-image 0.18.1
    Uninstalling scikit-image-0.18.1:
      Successfully uninstalled scikit-image-0.18.1
ERROR: pip's dependency resolver does not currently take into account all the packages that are installed. This behaviour is the source of the following dependency conflicts.
imgaug 0.4.0 requires opencv-python-headless, which is not installed.
You should consider upgrading via the '/opt/conda/bin/python3.8 -m pip install --upgrade pip' command.


In [ ]:
!pip list

In [10]:
%%bash
apt-get install libgdal-dev -y
apt-get install python-gdal -y
apt-get install python-numpy python-scipy -y
pip install requests

E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
E: Could not open lock file /var/lib/dpkg/lock-frontend - open (13: Permission denied)
E: Unable to acquire the dpkg frontend lock (/var/lib/dpkg/lock-frontend), are you root?
You should consider upgrading via the '/opt/conda/bin/python3.8 -m pip install --upgrade pip' command.


In [11]:
conda install -c conda-forge gdal

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.10.1
  latest version: 4.10.3

Please update conda by running

    $ conda update -n base conda



## Package Plan ##

  environment location: /opt/conda

  added / updated specs:
    - gdal


The following packages will be UPDATED:

  ca-certificates                      2020.12.5-ha878542_0 --> 2021.5.30-ha878542_0
  certifi                          2020.12.5-py38h578d9bd_1 --> 2021.5.30-py38h578d9bd_0


Preparing transaction: done
Verifying transaction: done
Executing transaction: done

Note: you may need to restart the kernel to use updated packages.


## Model code

The following steps are taken from the example notebooks on original repo. 

In [12]:
import os




##Inference

In [13]:
pwd


'/lustre_scratch/karennaturescot'

In [ ]:
##if needed
os.chdir('/lustre_scratch/karennaturescot')

In [14]:
import numpy as np
import os
import sys
#import gdal
print(os.getcwd())

ROOT_DIR = os.path.abspath('/lustre_scratch/karennaturescot/mask_rcnn')
#lustre_scratch/karennaturescot/mask_rcnn
MAERL_DIR = os.path.abspath('/lustre_scratch/karennaturescot/mask_rcnn/samples/maerl')
DATA_DIR = os.path.abspath('/lustre_scratch/karennaturescot/Deep_learning_maerl')

#sys.path.append(ROOT_DIR)
os.chdir(ROOT_DIR)
#os.chdir(DATA_DIR)
print(os.getcwd())
from mrcnn import visualize
from mrcnn import utils
from mrcnn.model import log
from mrcnn.config import Config
import mrcnn.model as modellib

%matplotlib inline

#sys.path.append(MAERL_DIR)

os.chdir(MAERL_DIR)
#import maerl_mageo_nucleiConfig
#import maerl_mageo_v2
import maerl_tiny
#import maerl_mageo
#import maerl_mageo_jpg
print(os.getcwd())


/lustre_scratch/karennaturescot
/lustre_scratch/karennaturescot/mask_rcnn
/lustre_scratch/karennaturescot/mask_rcnn/samples/maerl


In [15]:
#new class in script not .py

#OUTPUT_DIR = os.path.abspath('/content/drive/MyDrive/Deep_learning_maerl/output')
OUTPUT_DIR = os.path.abspath('/lustre_scratch/karennaturescot/Deep_learning_maerl/output')
MODEL_DIR = os.path.join(OUTPUT_DIR, 'logs')

class inferconfig(maerl_tiny.MaerlShapesConfig):
  NAME = 'maerlinfer'

  #IMAGES_PER_GPU = 2
  IMAGES_PER_GPU = 1

  GPU_COUNT = 1

  NUM_CLASSES = 1 + 1  # Just maerl

  IMAGE_RESIZE_MODE = "pad64" #"pad64"#"square"# "pad64" #"none" #"crop" #"square"
 #note  that none didn't work
  #FPN_CLASSIF_FC_LAYERS_SIZE =256# 1024 # would also need changing if max dim is changed?
   
  USE_MINI_MASK = False
  
  #DETECTION_MIN_CONFIDENCE = 0

    # Non-maximum suppression threshold for detection
  #DETECTION_NMS_THRESHOLD =0.7 # 0.7 as for nucleus example 0.9 #
      # Non-max suppression threshold to filter RPN proposals.
    # You can increase this during training to generate more propsals.
  RPN_NMS_THRESHOLD = 0.9#0.3 v13# 0.9 v11#0.99v7 #0.9v5#0.3 v4# 0.7 #as for nucleus example 0.9
  PRE_NMS_LIMIT = 9000 #6000 V4
  #POST_NMS_ROIS_TRAINING = 1000
  POST_NMS_ROIS_INFERENCE =7000# 8000v7# 7000v6# 6000 v4 # 10002000# 1000
  DETECTION_MIN_CONFIDENCE = 0.0001# 0.00001 v13#0.0001 v10, v9 #0.001 v8 # 0.01 v7 #0.01 v6 #0.2 v5
  DETECTION_NMS_THRESHOLD =0.9 #0.5 v12#v11 0.9#0.99 v7#0.9v6# 0.1v5# 0.2 the overlap of non maximal bounding boxes above which they are discarded

      # Max number of final detections per image
  DETECTION_MAX_INSTANCES = 10600#10500 v9 #10000v6 #10000 v4# 500 #1000 # tried 20000 crashed, 12000crashed, 11000crashed, 10900crashed
  IMAGE_MAX_DIM =3712# 3648 #needs to be multple of 64?
        # square: Resize and pad with zeros to get a square image
    #         of size [max_dim, max_dim].
  IMAGE_MIN_DIM =  2752# 
inference_config = inferconfig()
inference_config.display()

##note - v12, v13 worse than 11,10,9 - use exisiting config to maximise number of classifications without crashing mageohub


Configurations:
BACKBONE                       resnet101
BACKBONE_STRIDES               [4, 8, 16, 32, 64]
BATCH_SIZE                     1
BBOX_STD_DEV                   [0.1 0.1 0.2 0.2]
COMPUTE_BACKBONE_SHAPE         None
DETECTION_MAX_INSTANCES        10600
DETECTION_MIN_CONFIDENCE       0.0001
DETECTION_NMS_THRESHOLD        0.9
FPN_CLASSIF_FC_LAYERS_SIZE     1024
GPU_COUNT                      1
GRADIENT_CLIP_NORM             5.0
IMAGES_PER_GPU                 1
IMAGE_CHANNEL_COUNT            3
IMAGE_MAX_DIM                  3712
IMAGE_META_SIZE                14
IMAGE_MIN_DIM                  2752
IMAGE_MIN_SCALE                0
IMAGE_RESIZE_MODE              pad64
IMAGE_SHAPE                    [3712 3712    3]
LEARNING_MOMENTUM              0.9
LEARNING_RATE                  0.001
LOSS_WEIGHTS                   {'rpn_class_loss': 1.0, 'rpn_bbox_loss': 1.0, 'mrcnn_class_loss': 1.0, 'mrcnn_bbox_loss': 1.0, 'mrcnn_mask_loss': 1.0}
MASK_POOL_SIZE                 14
MASK_SHAPE    

In [16]:

#model_path ="/content/drive/MyDrive/Deep_learning_maerl/output/logs/maerl20210406T1113/mask_rcnn_maerl_runII.h5"

model_path ="/lustre_scratch/karennaturescot/Deep_learning_maerl/output/logs/maerl20210729T1317/mask_rcnn_maerl_0150.h5"


# Recreate the model in inference mode
model = modellib.MaskRCNN(mode="inference", 
                          config=inference_config,
                          model_dir=MODEL_DIR)

# Get path to saved weights
# Either set a specific path or find last trained weights

#model_path = model.find_last()

# Load trained weights
print("Loading weights from ", model_path)
model.load_weights(model_path, by_name=True)

Instructions for updating:
Use fn_output_signature instead
Loading weights from  /lustre_scratch/karennaturescot/Deep_learning_maerl/output/logs/maerl20210729T1317/mask_rcnn_maerl_0150.h5
Re-starting from epoch 150


#load in image to test

In [17]:
##class functionality to load in jpg and tiff

import glob
import os
import re
from osgeo import gdal
import numpy as np
from osgeo import ogr
import sys
import logging
import random

def scale_to_8bit(pix):
  scaled = np.zeros(pix.shape, dtype=float)
  for b in range(len(pix)):
      scaled[b] = pix[b]/pix[b].max()
  
  return (scaled*255).astype('uint8')

class MaerlShapesTEST(utils.Dataset):
  def load_maerl(self, dataset_dir, skip_empty=False):
           
    if not os.path.exists(dataset_dir):
      logging.error('Path {} does not exist'.format(dataset_dir))
      return  
    
    image_dir = 'images'
    json_dir = 'geometries'
    print(image_dir)
    print(json_dir)
    print(dataset_dir)
    print(os.path.join(dataset_dir,image_dir))
    #image_files = glob.glob(os.path.join(dataset_dir, image_dir, '*.tif'))
    image_files = glob.glob(os.path.join(dataset_dir, image_dir, '*.tif'))+glob.glob(os.path.join(dataset_dir, image_dir, '*.JPG'))
    #image_files1 = glob.glob(os.path.join(dataset_dir, image_dir, '*.tif'))
    #print(image_files1)
    #image_files1 = glob.glob(os.path.join(dataset_dir, image_dir, '*.jpg'))
    #print(image_files1)
    print(image_files)
    
    image_ids = [int(re.findall(r'\d+', img)[0]) for img in image_files]
    print(image_ids)
    # image_ids = [int(re.findall(r'\d+', img)[2]) for img in image_files]
    # json = glob.glob(os.path.join(dataset_dir, json_dir, '*.geojson'))
    #json = glob.glob(os.path.join(dataset_dir, json_dir, '*.json'))
    #print(json[0])
    # json_ids = [int(re.findall(r'\d+', js)[2]) for js in json]
    #json_ids = [int(re.findall(r'\d+', js)[0]) for js in json]
    #print(json_ids[0])
    
    if os.path.exists(os.path.join(dataset_dir, json_dir)):      
      json = glob.glob(os.path.join(dataset_dir, json_dir, '*.json'))
      json_ids = [int(re.findall(r'\d+', js)[0]) for js in json]
      
      for i, id_ in enumerate(image_ids):
        src = gdal.Open(image_files[i])
        if has_zero_instances(json[json_ids.index(id_)]):
          if skip_empty:
            continue # zero instance image

        self.add_image(
          "maerl",
          image_id=id_,
          path=image_files[i],
          width=3712 ,# adjusted for padding without distortion,
          height=2752, #adjusted for padding without distortion,
          polygons=json[json_ids.index(id_)],
          geotransform=src.GetGeoTransform()
        )
    else:
      for i, id_ in enumerate(image_ids):
        src = gdal.Open(image_files[i])
        self.add_image(
          "maerl",
          image_id=id_,
          path=image_files[i],
          #width=src.RasterXSize,
          #height=src.RasterYSize,
          width=3712, # adjusted for padding without distortion,
          height=2752, #adjusted for padding without distortion,
          geotransform=src.GetGeoTransform()
        )
    self.add_class('maerl', 1, 'maerl')

    #self.add_class('spacenet', 1, 'Building')
   #for i, id_ in enumerate(image_ids):
    #  src = gdal.Open(image_files[i])
    #  self.image_info[i] = {
       #   'path': image_files[i],
        #  'polygons': json[json_ids.index(id_)],
        #  'height': src.RasterYSize,
         # 'width': src.RasterXSize,
        #  'geotransform': src.GetGeoTransform(),
        #  'source': "maerlsource",
     #    'id': i,
 #    } #id needed for prepare data method

    #image_ids.sort()
    
    #self.num_Images = len(self.image_info)
    #self.nrange = np.arange(self.num_Images)
    
    
    #new_image_info = []
    #for i in self.nrange:
      #new_image_info.append(self.image_info.get(i))

    #self.image_info = new_image_info
    
    
    
    
    
    #self._image_ids = image_ids
    
  def load_image(self, image_id):
    img = gdal.Open(self.image_info[image_id]['path'])
    byte_img = scale_to_8bit(img.ReadAsArray())
    padrows = np.ones((3,16, 3648),dtype=np.uint8)
    padcolumns=np.ones((3,2752,64),dtype=np.uint8)
    byte_img = np.append(byte_img,padrows,axis=1)
    byte_img = np.append(byte_img, padcolumns,axis =2)
    
    return byte_img.transpose(1, 2, 0)
  
  #from balloon
  def image_reference(self, image_id):
      """Return the id of the image."""
      info = self.image_info[image_id]
      if info["source"] == "maerl":
          return info["id"]
      else:
          super(MaerlShapes, self).image_reference(image_id)
          
 
  def load_mask(self, image_id):
    '''Load instance masks for the given image.
      
      Returns:
        masks: A bool ndarray of binary masks with
          shape [h, w, instance count]
        class_ids: a 1d ndarray of class IDs of the
          instance masks.
    '''
    image_info = self.image_info[image_id]
    instances = []
    if 'polygons' in image_info:
      feats = load_as_features(image_info['polygons'])
      lyr = feats.GetLayer()

      # Rasterize each feature as a new instance
      driver = gdal.GetDriverByName('MEM')
      for feat in lyr:
        # Each feature is a new layer
        outdriver = ogr.GetDriverByName('MEMORY')
        out = outdriver.CreateDataSource('memData')

        #tmp = outdriver.Open('memData', 1)
        out_layer = out.CreateLayer('layer', geom_type=ogr.wkbPolygon)
        out_layer.CreateFeature(feat)

        mem_raster = driver.Create(
            '',
            image_info['width'],
            image_info['height'],
            gdal.GDT_Byte
        )
        mem_raster.SetGeoTransform(image_info['geotransform'])
        gdal.RasterizeLayer(mem_raster, [1], out_layer, burn_values=[1])

        instances.append(mem_raster.GetRasterBand(1).ReadAsArray())
    
    if len(instances) !=0:
      mask = np.stack(instances, axis=2).astype(np.bool)

      # All the same class
      classes = np.ones(mask.shape[-1], dtype=np.int32)
      
      return mask, classes

    else:
      # Call super class to return an empty mask
      return super(MaerlShapesTEST, self).load_mask(image_id)




In [22]:
inferdataset1 = MaerlShapesTEST()
INFER_DIR= os.path.abspath('/lustre_scratch/karennaturescot/Deep_learning_maerl/infer')

#"/lustre_scratch/karennaturescot/Deep_learning_maerl/output/logs/maerl20210611T1108/mask_rcnn_maerl_0050.h5"
inferdataset1.load_maerl(INFER_DIR, skip_empty=True)
inferdataset1.prepare() #previously called within split, therefore need to call separately
print("loaded data")
print(inferdataset1.image_info)
# Split
#dataset_train, dataset_test = dataset.split_train_test(proportion_train=0.7)
print("Infer image Count: {}".format(len(inferdataset1.image_ids)))
print(inferdataset1.image_ids)
#print("Class Count: {}".format(dataset_train.num_classes))
#for i, info in enumerate(dataset_train.class_info):
#    print("{:3}. {:50}".format(i, info['name']))

#print("Test image Count: {}".format(len(inferdataset.image_ids)))
#print("Class Count: {}".format(dataset_test.num_classes))
#for i, info in enumerate(dataset_test.class_info):
#    print("{:3}. {:50}".format(i, info['name']))

print(inferdataset1.image_ids)

images
geometries
/lustre_scratch/karennaturescot/Deep_learning_maerl/infer
/lustre_scratch/karennaturescot/Deep_learning_maerl/infer/images
['/lustre_scratch/karennaturescot/Deep_learning_maerl/infer/images/IMG_6951.JPG', '/lustre_scratch/karennaturescot/Deep_learning_maerl/infer/images/IMG_6952.JPG', '/lustre_scratch/karennaturescot/Deep_learning_maerl/infer/images/IMG_6953.JPG']
[6951, 6952, 6953]
loaded data
[{'id': 6951, 'source': 'maerl', 'path': '/lustre_scratch/karennaturescot/Deep_learning_maerl/infer/images/IMG_6951.JPG', 'width': 3712, 'height': 2752, 'geotransform': (0.0, 1.0, 0.0, 0.0, 0.0, 1.0)}, {'id': 6952, 'source': 'maerl', 'path': '/lustre_scratch/karennaturescot/Deep_learning_maerl/infer/images/IMG_6952.JPG', 'width': 3712, 'height': 2752, 'geotransform': (0.0, 1.0, 0.0, 0.0, 0.0, 1.0)}, {'id': 6953, 'source': 'maerl', 'path': '/lustre_scratch/karennaturescot/Deep_learning_maerl/infer/images/IMG_6953.JPG', 'width': 3712, 'height': 2752, 'geotransform': (0.0, 1.0, 0.

In [23]:
import PIL
from PIL import Image
from PIL import ImageOps


for image_id,idx in enumerate(inferdataset1.image_ids, start =0):
    print(image_id)
    inputfp= [a_dict["path"] for a_dict in inferdataset1.image_info][image_id]
    print(inputfp)
    lengthinputfp = len(inputfp)
    leninputname = lengthinputfp - 65
    outputname = 'infer_' + inputfp[(leninputname*-1):]
    ##v8 crashed, only output for a single image rather than 2 that were input and reached limit on memory while running
    print("Output file name:")
    print(outputname)
    OUTPUT_DIR = os.path.abspath('/lustre_scratch/karennaturescot/Deep_learning_maerl/output/images')
    outputfp=os.path.join(OUTPUT_DIR,outputname)
    print(outputfp)
    
    image,image_meta, gt_class_id, gt_bbox, gt_mask = \
        modellib.load_image_gt(inferdataset1, inference_config, image_id)
    info = inferdataset1.image_info[image_id]
    print("image ID:{}.{}({}){}".format(info["source"],info["id"],image_id, inferdataset1.image_reference(image_id)))
#run object detection
    results = model.detect([image],verbose =1)
    r = results[0]
    outarray =np.sum(r['masks'],axis=2)
    outarray = np.where(outarray==0,outarray, 1)
    outarray = outarray.astype(np.uint8)
    #edit so edge pixels removed 
    image = Image.fromarray(outarray)
    image = ImageOps.crop(image, (0,0,64,16)) 
    #create blank output array of zeros
    print(outarray.shape)
    image.save(outputfp)

#outarray = np.zeros([r['masks'].shape[0],r['masks'].shape[1],1])
#print(outarray.shape)
#print(outarray[:3,:3,:])
#print(np.sum(r['masks'][:,:,:100]))
#print(outarray)
#outarray=np.sum(r['masks'], axis=2)
#print(outarray.shape)
#print(outarray)
#print(outarray.max())
#outarray = np.where(outarray ==0, outarray, 1)
#outarray= outarray.astype(np.uint8)
#print(outarray.max())

#for i in enumerate (r['masks'].shape[2]:

#use PIL for imagery that is not georeferenced

                    

0
/lustre_scratch/karennaturescot/Deep_learning_maerl/infer/images/IMG_6951.JPG
Output file name:
infer_IMG_6951.JPG
/lustre_scratch/karennaturescot/Deep_learning_maerl/output/images/infer_IMG_6951.JPG


image ID:maerl.6951(0)6951
Processing 1 images
image                    shape: (2752, 3712, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 2752, 3712, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3712.00000  int64
anchors                  shape: (1, 2551362, 4)       min:   -0.13160  max:    1.10834  float32


/opt/conda/lib/python3.8/site-packages/tensorflow/python/keras/engine/training.py:2325: UserWarning: `Model.state_updates` will be removed in a future version. This property should not be used in TensorFlow 2.0, as `updates` are applied automatically.
  warnings.warn('`Model.state_updates` will be removed in a future version. '


(2752, 3712)
1
/lustre_scratch/karennaturescot/Deep_learning_maerl/infer/images/IMG_6952.JPG
Output file name:
infer_IMG_6952.JPG
/lustre_scratch/karennaturescot/Deep_learning_maerl/output/images/infer_IMG_6952.JPG


image ID:maerl.6952(1)6952
Processing 1 images
image                    shape: (2752, 3712, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 2752, 3712, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3712.00000  int64
anchors                  shape: (1, 2551362, 4)       min:   -0.13160  max:    1.10834  float32
(2752, 3712)
2
/lustre_scratch/karennaturescot/Deep_learning_maerl/infer/images/IMG_6953.JPG
Output file name:
infer_IMG_6953.JPG
/lustre_scratch/karennaturescot/Deep_learning_maerl/output/images/infer_IMG_6953.JPG


image ID:maerl.6953(2)6953
Processing 1 images
image                    shape: (2752, 3712, 3)       min:    0.00000  max:  255.00000  uint8
molded_images            shape: (1, 2752, 3712, 3)    min: -123.70000  max:  151.10000  float64
image_metas              shape: (1, 14)               min:    0.00000  max: 3712.00000  int64
anchors                  shape: (1, 2551362, 4)       min:   -0.13160  max:    1.10834  float32
(2752, 3712)
